In [9]:
# 方法一：在某路径下，训练数据与验证数据分开放在两个文件夹内，同时路径下还有train.txt与test.txt文件（每行为‘路径 类别’）

# 自定义Dataset类，要求图片与txt文件按要求放置！！
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.autograd import Variable  

In [ ]:
def default_loader(path):
    return Image.open(path).convert('RGB')


class MyDataset(Dataset):
    def __init__(self, txt, transform=None, target_transform=None, loader=default_loader):
        fh = open(txt, 'r')
        imgs = []
        for line in fh:
            line = line.strip('\n')
            line = line.rstrip()
            words = line.split()
            imgs.append((words[0],int(words[1][7]))) # 看情况修改
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = self.loader(fn)
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

In [2]:
# 路径下放置train test文件夹保存train test图片+train.txt test.txt文本文件，每行为‘路径+类别’
root = 'C:/Users/jxjsj/Desktop/JupyterHome/Data/fashion/' 

# 数据保存到两个数据加载器中
train_data = MyDataset(txt=root+'train.txt', transform=transforms.ToTensor())
train_data_loader = DataLoader(train_data, batch_size=200,shuffle=True) # batch_size自定义
test_data = MyDataset(txt=root+'test.txt', transform=transforms.ToTensor())
test_data_loader = DataLoader(test_data, batch_size=200,shuffle=True)  

In [7]:
class reluCNNet(torch.nn.Module):
    def __init__(self):
        super(reluCNNet, self).__init__()
        self.conv1 = torch.nn.Sequential( # 28
            torch.nn.Conv2d(3, 32, 5, 1, 2), # 28
            torch.nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True), # BN 处理
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2) # 14
        ) 
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 5, 1, 0), # 10
            torch.nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1, affine=True), # BN 处理
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2) # 5
        )
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 32, 5, 1, 0), # 1
            torch.nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True), # BN 处理
            torch.nn.ReLU()
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(32, 10)
        )

    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        res = conv3_out.view(conv3_out.size(0), -1)
        out = self.dense(res)
        return out

In [11]:
use_gpu = True

# model = reluCNNet()

if use_gpu:
    model = model.cuda()

optimizer = torch.optim.Adam(model.parameters())
loss_func = torch.nn.CrossEntropyLoss()

for epoch in range(10):
    print('epoch {}'.format(epoch + 1))
    # training-----------------------------
    model.train()
    train_acc = 0.
    L_train_pred = []
    L_train_real = []
    for step, (batch_x, batch_y) in enumerate(train_data_loader):
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        
        if use_gpu:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = torch.max(out, 1)[1]
        num_correct = (pred == batch_y).sum()
        train_acc += num_correct.data
        
#         L_train_pred += pred.cpu().numpy().tolist()
#         L_train_real += batch_y.cpu().numpy().tolist()
        
        if step%10 ==0:
            print('Step:',step+1,'Finished!')
    print('Train Acc: {:.6f}'.format(train_acc.cpu().numpy() / (len(train_data))))
#     print(classification_report(L_train_real,L_train_pred))

    # evaluation--------------------------------
    model.eval()
    with torch.no_grad():
        eval_acc = 0.
        L_val_pred = []
        L_val_real = []
        for batch_x, batch_y in test_data_loader:
            batch_x, batch_y = Variable(batch_x), Variable(batch_y)

            if use_gpu:
                batch_x = batch_x.cuda()
                batch_y = batch_y.cuda()

            out = model(batch_x)
            loss = loss_func(out, batch_y)
            
            pred = torch.max(out, 1)[1]
            num_correct = (pred == batch_y).sum()
            eval_acc += num_correct
            
#             L_val_pred += pred.cpu().numpy().tolist()
#             L_val_real += batch_y.cpu().numpy().tolist()
            
        print('Test Acc: {:.6f}'.format(eval_acc.cpu().numpy() / (len(test_data))))
#         print(classification_report(L_val_real,L_val_pred))

epoch 1
Step: 1 Finished!
Step: 11 Finished!
Step: 21 Finished!
Step: 31 Finished!
Step: 41 Finished!
Step: 51 Finished!
Step: 61 Finished!
Step: 71 Finished!
Step: 81 Finished!
Step: 91 Finished!
Step: 101 Finished!
Step: 111 Finished!
Step: 121 Finished!
Step: 131 Finished!
Step: 141 Finished!
Step: 151 Finished!
Step: 161 Finished!
Step: 171 Finished!
Step: 181 Finished!
Step: 191 Finished!
Step: 201 Finished!
Step: 211 Finished!
Step: 221 Finished!
Step: 231 Finished!
Step: 241 Finished!
Step: 251 Finished!
Step: 261 Finished!
Step: 271 Finished!
Step: 281 Finished!
Step: 291 Finished!
Train Acc: 0.905283
Test Acc: 0.895200
epoch 2
Step: 1 Finished!
Step: 11 Finished!
Step: 21 Finished!
Step: 31 Finished!
Step: 41 Finished!
Step: 51 Finished!
Step: 61 Finished!
Step: 71 Finished!
Step: 81 Finished!
Step: 91 Finished!
Step: 101 Finished!
Step: 111 Finished!
Step: 121 Finished!
Step: 131 Finished!
Step: 141 Finished!
Step: 151 Finished!
Step: 161 Finished!
Step: 171 Finished!
Step: 1

In [ ]:
import os
from skimage import io
import torchvision.datasets.mnist as mnist

root="C:/Users/jxjsj/Desktop/JupyterHome/Data/fashion/"
# train_set = (
#     mnist.read_image_file(os.path.join(root, 'train-images-idx3-ubyte')),
#     mnist.read_label_file(os.path.join(root, 'train-labels-idx1-ubyte'))
#         )
# test_set = (
#     mnist.read_image_file(os.path.join(root, 't10k-images-idx3-ubyte')),
#     mnist.read_label_file(os.path.join(root, 't10k-labels-idx1-ubyte'))
#         )
# print("training set :",train_set[0].size())
# print("test set :",test_set[0].size())

def convert_to_img(train=True):
    if(train):
        f=open(root+'train.txt','w')
        data_path=root+'/train/'
        if(not os.path.exists(data_path)):
            os.makedirs(data_path)
        for i, (img,label) in enumerate(zip(train_set[0],train_set[1])): # 迭代方式要改变
            img_path=data_path+str(i)+'.jpg'
            io.imsave(img_path,img.numpy()) # 不保存图片则不用这段代码
            f.write(img_path+' '+str(label)+'\n')
        f.close()
    else:
        f = open(root + 'test.txt', 'w')
        data_path = root + '/test/'
        if (not os.path.exists(data_path)):
            os.makedirs(data_path)
        for i, (img,label) in enumerate(zip(test_set[0],test_set[1])): # 迭代方式要改变
            img_path = data_path+ str(i) + '.jpg'
            io.imsave(img_path, img.numpy()) # 不保存图片则不用这段代码
            f.write(img_path + ' ' + str(label) + '\n')
        f.close()

convert_to_img(True)
convert_to_img(False)